In [20]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import yfinance as yf
#IBOV
net = yf.Ticker('^BVSP')
net
# Set the timeframe you are interested in viewing.
net_historical = net.history(start="2018-01-2", end="2020-12-11", interval="1d")
# Create a new DataFrame called signals, keeping only the 'Date' & 'Close' columns.
signals_df = net_historical.drop(columns=['Volume','Dividends', 'Stock Splits'])

In [21]:
# Funções para criar target

def target_Close_maior_Open(row, delta):
  if row['Close'] > (1+delta)*row['Open']:
    target = 1
  else:
    target = 0
  return target

def target_High_maior_Open(row, delta):
  if row['High'] > (1+delta)*row['Open']:
    target = 1
  else:
    target = 0
  return target

def target_High_igual_Open(row):
  if row['High'] == row['Open']:
    target = 1
  else:
    target = 0
  return target

def target_High_menor_Open(row, delta):
  if row['High'] <= (1+delta)*row['Open']:
    target = 1
  else:
    target = 0
  return target

In [22]:
# criar target

signals_df['target_CO'] = signals_df.apply(lambda x: target_Close_maior_Open(x, delta=0.01), axis=1)
signals_df['target_HO'] = signals_df.apply(lambda x: target_High_maior_Open(x, delta=0.01), axis=1)


In [23]:
# Set the short window and long windows
short_window = 50
long_window = 100
# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['Close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)
# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()
# Print the DataFrame
signals_df.tail(10)

,Open,High,Low,Close,target_CO,target_HO,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,,,,,,
2020-11-27,110229.0,111603.0,110161.0,110575.0,0,1,100238.78,100939.81,0.0,0.0
2020-11-30,110598.0,110934.0,108829.0,108888.0,0,0,100423.02,101051.08,0.0,0.0
2020-12-01,108897.0,111840.0,108897.0,111335.0,1,1,100647.76,101166.73,0.0,0.0
2020-12-02,111402.0,112315.0,110579.0,111814.0,0,0,100918.24,101293.27,0.0,0.0
2020-12-03,111874.0,113377.0,111874.0,112919.0,0,1,101236.80,101422.14,0.0,0.0
2020-12-04,112294.0,113864.0,112294.0,113682.0,1,1,101564.56,101571.99,0.0,0.0
2020-12-07,113751.0,114531.0,112629.0,113625.0,0,0,101922.36,101703.84,1.0,1.0
2020-12-08,113590.0,114381.0,112820.0,113571.0,0,0,102253.54,101821.64,1.0,0.0
2020-12-09,113799.0,114020.0,112567.0,112722.0,0,0,102568.00,101943.33,1.0,0.0


In [25]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['target_CO'] == 1.0]['Close'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='IBOV',
    width=1000,
    height=400
)
# Visualize entry position relative to close price
entry = signals_df[signals_df['target_CO'] == 1.0]['Open'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='IBOV',
    width=1000,
    height=400
)
# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightpink',
    ylabel='IBOV',
    width=1000,
    height=400
)

security_open = signals_df[['Open']].hvplot(
    line_color='lightgreen',
    ylabel='IBOV',
    width=1000,
    height=400
)
# # Visualize moving averages
# moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )
# Overlay plots
entry_exit_plot = security_close * security_open * entry * exit
entry_exit_plot.opts(title='Close maior que Open', xaxis=None)

:Overlay
   .Curve.I       :Curve   [Date]   (Close)
   .Curve.II      :Curve   [Date]   (Open)
   .Scatter.Open  :Scatter   [Date]   (Open)
   .Scatter.Close :Scatter   [Date]   (Close)

In [26]:

# Visualize exit position relative to close price
exit = signals_df[signals_df['target_HO'] == 1.0]['High'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='IBOV',
    width=1000,
    height=400
)
# Visualize entry position relative to close price
entry = signals_df[signals_df['target_HO'] == 1.0]['Open'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='IBOV',
    width=1000,
    height=400
)
# Visualize close price for the investment
security_close = signals_df[['High']].hvplot(
    line_color='lightpink',
    ylabel='IBOV',
    width=1000,
    height=400
)

security_open = signals_df[['Open']].hvplot(
    line_color='lightgreen',
    ylabel='IBOV',
    width=1000,
    height=400
)
# # Visualize moving averages
# moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )
# Overlay plots
entry_exit_plot = security_close * security_open * entry * exit
entry_exit_plot.opts(title='High maior que Open', xaxis=None)

:Overlay
   .Curve.I      :Curve   [Date]   (High)
   .Curve.II     :Curve   [Date]   (Open)
   .Scatter.Open :Scatter   [Date]   (Open)
   .Scatter.High :Scatter   [Date]   (High)

In [27]:
close_ratio = (signals_df['Close']/signals_df[['Open']]).hvplot(
    line_color='lightgreen',
    ylabel='IBOV',
    width=1000,
    height=400
)

close_ratio.opts(xaxis=None)

KeyboardInterrupt: 

In [16]:
# Set initial capital
initial_capital = float(100000)
# Set the share size
share_size = 500
# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']
# Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()
# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Entry/Exit Position'].cumsum()
# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()
# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()
# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
# Print the DataFrame
signals_df.tail(10)

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2020-11-27,110575.0,100238.78,100939.81,0.0,0.0,0.0,0.0,0.0,-2615500.0,-2615500.0,0.000000,-27.155
2020-11-30,108888.0,100423.02,101051.08,0.0,0.0,0.0,0.0,0.0,-2615500.0,-2615500.0,0.000000,-27.155
2020-12-01,111335.0,100647.76,101166.73,0.0,0.0,0.0,0.0,0.0,-2615500.0,-2615500.0,0.000000,-27.155
2020-12-02,111814.0,100918.24,101293.27,0.0,0.0,0.0,0.0,0.0,-2615500.0,-2615500.0,0.000000,-27.155
2020-12-03,112919.0,101236.80,101422.14,0.0,0.0,0.0,0.0,0.0,-2615500.0,-2615500.0,0.000000,-27.155
2020-12-04,113682.0,101564.56,101571.99,0.0,0.0,0.0,0.0,0.0,-2615500.0,-2615500.0,0.000000,-27.155
2020-12-07,113625.0,101922.36,101703.84,1.0,1.0,500.0,500.0,56812500.0,-59428000.0,-2615500.0,0.000000,-27.155
2020-12-08,113571.0,102253.54,101821.64,1.0,0.0,500.0,0.0,56785500.0,-59428000.0,-2642500.0,0.010323,-27.425
2020-12-09,112722.0,102568.00,101943.33,1.0,0.0,500.0,0.0,56361000.0,-59428000.0,-3067000.0,0.160643,-31.670


In [17]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)
# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)
# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)
# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [18]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']
columns = ['Backtest']
# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

In [19]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0
target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev
portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,11.1241
Cumulative Returns,-20.32
Annual Volatility,16.6876
Sharpe Ratio,0.666609
Sortino Ratio,2.88016


In [20]:
# Initialize trade evaluation DataFrame with columns.
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

In [21]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']
    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss =  entry_portfolio_holding - exit_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'NET',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

In [22]:
price_df = signals_df[['Close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(title='Cloudflare', xaxis=None)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [23]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]